In [1]:
import sqlite3
import os
from pathlib import Path
from anthropic import Anthropic

key = os.environ["ANTHROPIC_API_KEY"] 
REPO_PATH = Path("/Users/koen/workspace/agentic-syftbox")
SCREENPIPE_PATH = REPO_PATH / "data" / "screenpipe" / "db.sqlite" 


In [2]:
# Connect to the SQLite database
conn = sqlite3.connect(SCREENPIPE_PATH)
conn.row_factory = sqlite3.Row

In [ ]:
    cursor = connection.cursor()
    # Query the audio_transcriptions table
    cursor.execute("SELECT * FROM audio_transcriptions")
    rows = [dict(x) for x in cursor.fetchall()]
    fragments = []

    for row in rows:
        speaker_id = row["speaker_id"]
        transcription = row["transcription"]
        
        fragment = f"Speaker {speaker_id}:   {transcription}"
        fragments.append(fragment)

    full_transcription = "\n".join(fragments)
    return [full_transcription]